# Google Earth Engine

**OPEN IN GOOGLE COLLAB OR INSTALL EARTHENGINE MODULE IN YOUR LOCAL ENVIRONMENT**

The following code is used to aggregate environmental parameters by-month by-county using Google Earth Engine.

First things first, authenticate EarthEngine with a google earth engine account. This will only need to be done the first time you run this script.

Run the following code cell **only once**. 

In [0]:
import ee

ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=bVXbj4Ytk2AxldpCqSUU3TG2X1V9m6NtY90s_yB78rk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/ywEQsE6UEJ-qAh73fZ0f_tIvHDoKCamZpHtnu261QzeVd8mpLYPzbAw

Successfully saved authorization token.


## Imports

In [0]:
import ee
import pandas
ee.Initialize()

## Function: imageParams
imageParams takes an ee.Image() object and returns a pandas dataframe with the mean values of the image for each feature in an ee.FeatureCollection object.

ee.Image is essentially a raster and ee.FeatureCollection is essentially a shapefile. 

**Image.reduceRegions()** does the majority of the work. It takes our image and our shapefile and it returns the shapefile with the mean value of each band of the image appended to each feature.

**.getInfo()** converts the information from an EE server-side object (inaccessible) to a client-side dictionary (accessible). It is frowned upon because it takes a comparatively long time, but it is perfect for this scenario. Reducing the number of .getInfo calls, in this case, results in an EE *memory limit exceeded* error.

Once the data is converted to client side, a simple for loop is used to extract the data from the nested dictionary.

In [0]:
def imageParams(image, shapefile):
    """
    Takes an image and a shapefile, returns a dataframe with the mean band values summarized by shapefile feature. 
    """
    #Gets the average of the band values in each feature in the iran asset. See: https://developers.google.com/earth-engine/reducers_reduce_region
    envFC=image.reduceRegions(shapefile, ee.Reducer.mean(), image.projection().nominalScale(), image.projection())
    envDict=envFC.getInfo()['features']

    #Now, features is a messy list of dictionaries. We need to extract the "properties" from each feature. 
    cols=[] #blank variable to store properties
    for feature in envDict:
        cols.append(feature['properties']) #adds the properties list to cols
    
    return pandas.DataFrame(cols) #creates a new pandas dataframe with the data in cols

## Function: getYearlyParams
This function mostly just serves up an image to imageParams. One image is required for each month.

**first=True**:
For image collections with only one image per month, the first image in the filtered collection can be used to speed up processing times. For those with daily collection (such as ndvi), the mean of the month of images is used. This is specified with the 'first' parameter. 

**return yearlyParams**:
The image is served up to imageParams() which returns a dataframe. This is appended to the yearlyParams dataframe, which is returned when each month of each year has been run. 

In [0]:
def getYearlyParams(collection, shapefile, startYear, endYear, first=True):
    """
    Returns the mean of each band in collection aggregated by region in shapefile for each month between startYear and endYear.
    -collection: an EarthEngine ImageCollection filtered to only the desired bands 
    -shapefile: an EarthEngine FeatureColleciton with regions to be summarized (must be named ADM2_EN)
    -startYear, endYear: The code will summarize these years, January-December
    -first: if true, uses the first image in each month of data. If false, uses the mean value of any images in the month.
    """
    yearlyParams=pandas.DataFrame(columns=['ADM2_EN']) #sets up an output dataframe
    
    #Iterates through each year and month in the startYear-endYear range
    for year in range(startYear, endYear+1):
        print(year)
        for month in range(1, 13):
            print(month, end=' ')
            if(first):
                #Filters the Image Collection object by year, and then by month, and then takes the first image in the resulting collection
                image=collection.filter(ee.Filter.calendarRange(year,year,'year')).filter(ee.Filter.calendarRange(month, month,'month'))\
                .first()
            else:
                #does the same as above, but takes the averages all the images in the collection if 'first' is set to False
                image=collection.filter(ee.Filter.calendarRange(year,year,'year')).filter(ee.Filter.calendarRange(month, month,'month'))\
                .mean()
            #With the resulting image, gets the imageParams and adds them to the output dataframe
            yearlyParams=yearlyParams.merge(imageParams(image, shapefile),\
                                            how='outer',\
                                            on='ADM2_EN',\
                                            suffixes=('', '_'+str(year)+str(month).zfill(2)))#zfill to make sure months are saved as '04' instead of '4'
    print("done")
    return yearlyParams

## Main

The code was run to aggregate monthly temperature, precipitation, and ndvi data by county. Elevation data was aggregated by county only. 


**Temperature and Precipitation**

The data collection is the ERA5 atmospheric reanalysis. It uses previous forecast data corrected with weather observations to create a gridded 0.25x0.25 degree raster of actual weather conditions. This version is aggregated by month. 

More info here: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

**NDVI**

The NOAA AHRR dataset was used for NDVI. This contains gridded daily NDVI values at a 0.05 degree resolution. 

More info here: https://www.ncdc.noaa.gov/cdr/terrestrial/normalized-difference-vegetation-index 

**Elevation**

The Japanese Aerospace Exploration Agency 30 meter digital elevation model was used for elevation data.

More info here: https://www.eorc.jaxa.jp/ALOS/en/aw3d30/index.htm 

In [0]:
iran = ee.FeatureCollection("users/tannerjohnson56/iran_admin").select("ADM2_EN")  #Loads the iran boundaries asset with only county name field kept

#Loads the satellite imagery
dem=ee.Image("JAXA/ALOS/AW3D30/V2_2").select('AVE_DSM')
ndvi=ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5").select('NDVI') 
moWeather = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select("mean_2m_air_temperature","total_precipitation") 

#For this example only year 2008 is used to save time
start=2008
end=2008
weatherParams=getYearlyParams(moWeather, iran, start, end)
ndviParams=getYearlyParams(ndvi, iran, start, end, False)
elevParams=imageParams(dem, iran)

allParams=weatherParams.merge(ndviParams, on='ADM2_EN').merge(elevParams, on='ADM2_EN')
display(allParams.head().rename(columns={'mean_y':'mean_elevation'}))
#Writing data to a .csv file
#allParams.to_csv(r'allParams.csv', index=False)

Temperature: degrees Kelvin

Precipitation: meters

Elevation: meters above sea level

NDVI is multiplied by 10,000 to have integer rather than float values.